In [4]:
import qcodes as qc
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
import qcodes.dataset.experiment_container as exc
import dash
import dash_core_components as dcc
import dash_html_components as html
from qcodes.dataset.data_set import load_by_id
from qcodes.dataset.data_export import get_data_by_id
from qcodes import load_experiment
from dash.dependencies import Input, Output, State
from plotly import tools
import plotly.io as pio
from plotly import graph_objs as go
from qcodes.instrument_drivers.rohde_schwarz import SGS100A
from time import sleep
from qcodes.dataset.measurements import Measurement
from scipy.optimize import curve_fit
import datetime
import scipy.fftpack
from scipy import signal
from scipy.signal import find_peaks
import pdfkit
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc

pyqtgraph plotting not supported, try "from qcodes.plots.pyqtgraph import QtPlot" to see the full error


In [5]:
configuration = qc.config
print(f'Using config file from {configuration.current_config_path}')
configuration['core']['db_location'] = './2019-07-08-frays-final.db'
print(f'Database location: {configuration["core"]["db_location"]}')
qc.initialise_database()

Using config file from //anaconda3/lib/python3.7/site-packages/qcodes/config/qcodesrc.json
Database location: ./2019-07-08-frays-final.db


In [6]:
#exc.experiments()

In [7]:
def normalizeCounts(countArray, num):
    reb_counts = np.squeeze(countArray)/np.mean(np.sort(np.squeeze(countArray))[-num:])
    return reb_counts

In [8]:
def plotdata(expt, files, normalize, plotcurrent=0, nPi = []): 
    "Attribute - expt - can either be 'counting' or 'odmr' or 'pulsedodmr' or 'rabi' or 'ramsey' or 'spinecho' or 'doubleecho' or 'nmr'.\
    If value of plotcurrent is 1 then it will plot the current data. If value of normalize is 1 then it will normalize 'odmr' and 'doubleecho' signal."
    if plotcurrent == 1:        
        filesize = np.size(files) + 1
    else:
        filesize = np.size(files)  
        
    plotfun = go.Figure()    
    for i in range(filesize):
        if plotcurrent == 1 and i == filesize-1:
            Data2 = exc.load_last_experiment()
            Data2 = Data2.last_data_set()
        else:    
            Data2 = exc.load_by_id(files[i])
        if expt == 'spinecho':
            x2 = 2*np.squeeze(Data2.get_data('Time'))
            y2 = np.squeeze(Data2.get_data('Rebased_Counts'))
            plotfun.layout = go.Layout(xaxis=dict(title='Time (ns)'),yaxis=dict(title='Counts'),title='Spin Echo')                  
        elif expt == 'doubleecho':
            if nPi == [] or nPi[i] == 1:
                x2 = 2*np.squeeze(Data2.get_data('Time'))
            else:
                x2 = nPi[i]*np.squeeze(Data2.get_data('Time'))
            y2ms0 = np.squeeze(Data2.get_data('Act_Counts'))
            y2ms1 = np.squeeze(Data2.get_data('Ref_Counts'))
            y2 = y2ms0 - y2ms1   
            plotfun.layout = go.Layout(xaxis=dict(title='Time (ns)'),yaxis=dict(title='Counts'),title='Spin Echo Double Measure')
            if normalize == 1:
                y2 = (y2 + max(y2))/(2*max(y2))
        elif expt == 'nmr':
            x2 = np.squeeze(Data2.get_data('Time'))
            y2ms0 = np.squeeze(Data2.get_data('Act_Counts'))
            y2ms1 = np.squeeze(Data2.get_data('Ref_Counts'))
            y2 = y2ms0 - y2ms1   
            plotfun.layout = go.Layout(xaxis=dict(title='Time (ns)'),yaxis=dict(title='Counts'),title='NMR')
            if normalize == 1:
                y2 = (y2 + max(y2))/(2*max(y2))
        elif expt == 'odmr':
            x2 = np.squeeze(Data2.get_data('Frequency'))
            y2 = np.squeeze(Data2.get_data('Counts'))
            plotfun.layout = go.Layout(xaxis=dict(title='Frequency (Hz)'),yaxis=dict(title='Counts'),title='ODMR')
            if normalize == 1:
                y2 = normalizeCounts(Data2.get_data('Counts'),50)
                layout = go.Layout(xaxis=dict(title='Frequency (Hz)'),yaxis=dict(title='Normalized Counts'),title='ODMR')
        elif expt == 'pulsedodmr':
            x2 = np.squeeze(Data2.get_data('Frequency'))
            y2 = np.squeeze(Data2.get_data('Rebased_Counts'))   
            plotfun.layout = go.Layout(xaxis=dict(title='Frequency (Hz)'),yaxis=dict(title='Counts'),title='Pulsed ODMR')
        elif expt == 'g2':
            x2 = np.squeeze(Data2.get_data('Time'))
            y2 = np.squeeze(Data2.get_data('Norm_Counts'))
            plotfun.layout = go.Layout(xaxis=dict(title='Time dif'), yaxis=dict(title='Normalised Counts'), title='g2 Dip')
        else:
            x2 = np.squeeze(Data2.get_data('Time'))
            y2 = np.squeeze(Data2.get_data('Rebased_Counts'))     
            plotfun.layout = go.Layout(xaxis=dict(title='Time (ns)'),yaxis=dict(title='Counts'),title='Rabi')
        if plotcurrent == 1 and i == filesize-1:
            plotfun.add_scatter(x = x2, y = y2, name = 'Recent Data', mode='lines+markers') 
        else:       
            plotfun.add_scatter(x = x2, y = y2, name = files[i], mode='lines+markers') 
    return plotfun

In [9]:
experimentsList = [exp.name for exp in exc.experiments()]
types = [{'label':'Counting', 'value': 'counting'},
         {'label':'ODMR', 'value': 'odmr'},
         {'label':'Pulsed ODMR', 'value': 'pulsedodmr'},
         {'label':'Rabi', 'value': 'rabi'},
         {'label':'Ramsey', 'value': 'ramsey'},
         {'label':'Spin Echo', 'value': 'spinecho'},
         {'label':'Double Echo', 'value': 'doubleecho'},
         {'label':'NMR', 'value': 'nmr'}]
dataNeeded = {'odmr': ['Frequency','Counts'], 
              'pulsedodmr': ['Frequency','Rebased_Counts'], 
              'spinecho': ['Time','Rebased_Counts'],
              'doubleecho': ['Time','Act_Counts','Ref_Counts'],
              'nmr': ['Time','Act_Counts','Ref_Counts'],
              'counting': ['Time','Rebased_Counts'],
              'rabi': ['Time','Rebased_Counts'],
              'ramsey':['Time','Rebased_Counts']}
analysisTypes = {'odmr': [['Lorentzian','lorentz']], 
                 'pulsedodmr': [['Lorentzian','lorentz']],
                 'spinecho': [['Stretched Exponential','exponent'],['Fourier Transform','fourier']],
                 'doubleecho': [['Stretched Exponential','exponent'],['Fourier Transform','fourier']],
                 'nmr': [['Fourier Transform','fourier']],
                 'counting': [['Not Available!','none']],
                 'rabi': [['Damped Sine','sine']],
                 'ramsey': [['Fourier Transform','fourier']]}
lastSet = (qc.load_last_experiment()).last_data_set()
all_runs = [{'label': str(load_by_id(a)).split('@')[0], 'value': a} for a in range(1,lastSet.run_id)]

In [12]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.config.suppress_callback_exceptions = True

app.layout = html.Div([
    dcc.Store(id='tab-1-data', storage_type='session'),
    dcc.Store(id='tab-2-data', storage_type='session'),
    dcc.Tabs(id="tabs", value='tab-1', children=[dcc.Tab(label='Graph', value='tab-1'),
                                                 dcc.Tab(label='Analyse', value='tab-2'),
    ]),
    html.Div(id='tabs-content')],  
    style={'columnCount': 1})

@app.callback(Output('tabs-content', 'children'),
              [Input('tabs', 'value')])
def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            html.H2(children='Graph Data'),
            html.Div([
                html.Label('1) Experiment'), 
                dcc.Dropdown(id='experiment', 
                     options=[{'label': experimentsList[o], 'value': o+1} 
                              for o in range((len(experimentsList)))]),
                html.Label('2) Runs'), 
                dcc.Dropdown(id='runs', multi=True),
                html.Label('3) Data Type'),
                dcc.Dropdown(id='data-type', options = types),
                html.P(id='type-err', style={'color': 'red'})],
                style={'width': '48%', 'display': 'inline-block'}), 
            html.Div([
                html.Label('4) Extra Runs (Optional)'), 
                dcc.Dropdown(id='extra-runs',options = all_runs, multi=True),
                html.P(id='run-err', style={'color': 'red'}),
                html.Label('5) Normalise?'), 
                dcc.RadioItems(id='normalise', options = [{'label': 'Yes', 'value': 1}, 
                                                          {'label': 'No', 'value': 0}], value = 1),
                html.Button(id='submit-button', children='Plot!')], 
                style={'width': '48%', 'float': 'right', 'display': 'inline-block'}),
        html.Hr(),
        dcc.Graph(style={'height': '700px'},id='graph')
        ])
    elif tab == 'tab-2':
        return html.Div([
            html.H2(children='Analyse Data'),
            html.Div([
                html.Label('1) Runs'),
                dcc.Dropdown(id='analysis-runs',options = all_runs, multi=True),
                html.Label('2) Data Type'),
                dcc.Dropdown(id='data-type', options = types)],
                style={'width': '48%', 'display': 'inline-block'})    
        ], style={'columnCount': 1})

@app.callback(
    Output('runs', 'options'),
    [Input('experiment', 'value')])
def update_run(selected_experiment):
    if selected_experiment is None:
        raise PreventUpdate
    exper = load_experiment(selected_experiment)
    return [{'label': str(exper.data_set(a+1)).split('@')[0], 
             'value': exper.data_set(a+1).run_id} for a in range(exper.last_counter)]

@app.callback(
    [Output('graph', 'figure'), Output('type-err','children'), 
     Output('run-err','children')],
    [Input('submit-button','n_clicks')],
    [State('runs','value'), State('data-type','value'), 
     State('extra-runs','value'), State('normalise', 'value')])
def update_graph(clicks, selected_runs, selected_type, extra_runs, normalised):
    if selected_runs is None:
        raise PreventUpdate
    for a in selected_runs:
        params = str(load_by_id(a).parameters).split(',')
        for b in dataNeeded[selected_type]:
            if (b in params) is False:
                return dash.no_update, 'Wrong Data Type!', dash.no_update
    if extra_runs != None:
        for c in extra_runs:
            params_2 = str(load_by_id(c).parameters).split(',')
            for d in dataNeeded[selected_type]:
                if (d in params_2) is False:
                    return dash.no_update, dash.no_update, 'Wrong Run!'
        selected_runs.extend(extra_runs)
    totaldata = plotdata(selected_type, selected_runs, normalised)
    return totaldata, None, None

In [13]:
if __name__ == '__main__':
    app.run_server(debug=False, port=8076, threaded=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8076/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Aug/2019 11:46:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Aug/2019 11:46:58] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1566376284 HTTP/1.1" 200 -
127.0.0.1 - - [30/Aug/2019 11:46:58] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.min.js?v=0.7.0&m=1567160754 HTTP/1.1" 200 -
127.0.0.1 - - [30/Aug/2019 11:46:58] "GET /_dash-component-suites/dash_core_components/dash_core_components.min.js?v=1.1.1&m=1566376285 HTTP/1.1" 200 -
127.0.0.1 - - [30/Aug/2019 11:46:58] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1566376284 HTTP/1.1" 200 -
127.0.0.1 - - [30/Aug/2019 11:46:58] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1566376284 HTTP/1.1" 200 -
127.0.0.1 - - [30/Aug/2019 11:46:58] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.1.1&m=1566376285 HTTP/1.1